## Meals Count Backend Utilities  
  
This notebook contains Python functionality to be incorporated into modules part of the [MealsCount](https://github.com/opensandiego/mealscount-backend) backend. **MealsCount** is an open-source project currently underway at [**OpenSanDiego**](https://opensandiego.org/), a [Code for America](https://www.codeforamerica.org/) brigade.  
  
**Note**: See [**[1]**](backend_utils_test.ipynb) for a detailed explanation of the core functionality, including intermediate outputs based on sample test data.   

In [1]:
import sys
import os
import pandas as pd
import numpy as np

from abc import ABC, abstractmethod

In [2]:
#
# GLOBAL CONSTANTS (DO NOT MODIFY)
#

# these are used to identify data rows 
# level 1 header
DATA_L1_HDR_KEYS = ['Non-Charter School(s)','Charter School(s)']
# level 2 header
DATA_L2_HDR_KEYS = ['School Code','School Name','Total Enrollment','Free & Reduced Meal Program: 181/182',
                    'Foster','Homeless(1)','Migrant Program: 135','Direct Certification',
                    'Unduplicated Eligible Free/Reduced Meal Counts','EL Funding Eligible (2)',
                    'Total Unduplicated FRPM/EL Eligible (3)']
# keywords for aggregated rows
DATA_SUM1_KEYS = ['TOTAL - Selected Schools']
DATA_SUM2_KEYS = ['TOTAL LEA']

# these are used for recoding header names/col values where applicable
DATA_L1_HDR_DICT = {'Non-Charter School(s)':'non-charter','Charter School(s)':'charter'}
DATA_L2_HDR_DICT = {'School Code':'school_code','School Name':'school_name','Total Enrollment':'total_enrolled',
                    'Free & Reduced Meal Program: 181/182':'frpm','Foster':'foster','Homeless (1)':'homeless',
                    'Migrant Program: 135':'migrant','Direct Certification':'direct_cert',
                    'Unduplicated Eligible Free/Reduced Meal Counts':'frpm_nodup',
                    'EL Funding Eligible (2)':'el', 'Total Unduplicated FRPM/EL Eligible (3)':'frpm_el_nodup'}
DATA_SUM_DICT = {'TOTAL - Selected Schools':'total','TOTAL LEA':'total'}

# these are used for recoding specific col values
INVALID_SCHOOL_CODE = 9999999
ALL_SCHOOL_TYPE = 'lea'

# these are used to identify metadata rows
METADATA_KEYS = ['Academic Year','View','As Of','Gender','School Type','School','User ID',
                 'Created Date','LEA']
# these are used to identify cols corresponding to 
# metadata key-value pairs
METADATA_KEY_COLS = [0,2,4]
METADATA_VAL_COLS = [1,3,5]

In [3]:
class mcSchoolDistInput(ABC):
    """
    Base class for school district input. 
    """
    d_df = pd.DataFrame()
    md_dict = {}
    
    def __init__(self):                
        pass
    
    @abstractmethod
    def to_frame(self):
        pass
    
    @abstractmethod
    def metadata(self):
        pass

In [4]:
#
# Function to extract and return a dataframe from the input
# dataframe and the row and col indices specified. Additionally
# a column for school type is added with the specified value as
# well as a column (called 'index') with the original row indices.
#
def extract_df(df,row_idx,col_idx,school_type):
    
    data = df.loc[row_idx,:].values
    cols = df.loc[col_idx].values
    ext_df = pd.DataFrame(data=data,columns=cols)        
    
    ext_df['school_type'] = school_type 
    ext_df['index'] = row_idx
    
    ext_df.dropna(axis=1,how='all',inplace=True)
    ext_df.dropna(axis=0,how='all',inplace=True)
    
    return ext_df

In [5]:
# FIXME: refactor code in here
def parseXL(self,xlfile):
    
    try:
        xl = pd.ExcelFile(xlfile)                
        tmpdf = xl.parse(xl.sheet_names[0])       
        
        # get the indices for the rows where the L1 headers are present
        data_l1 = tmpdf.index[tmpdf[tmpdf.isin(DATA_L1_HDR_KEYS)].notnull().any(axis=1)].tolist()
        # get indices for rows where the L2 headers are present
        # these will indicate the beginning of data 
        data_l2_begin = tmpdf.index[tmpdf[tmpdf.isin(DATA_L2_HDR_KEYS)].notnull().any(axis=1)].tolist()
        # get indices for the rows where the misc headers are present
        # these will indicate the end of data
        data_l2_end = tmpdf.index[tmpdf[tmpdf.isin(DATA_SUM1_KEYS)].notnull().any(axis=1)].tolist()
        # get indices for any other keys that are part of data
        data_other = tmpdf.index[tmpdf[tmpdf.isin(DATA_SUM2_KEYS)].notnull().any(axis=1)].tolist()

        # generate indices of non-data rows 
        metadata_idx = list(range(0,data_l1[0]))
        n = len(DATA_L1_HDR_KEYS)

        # TODO: malformed files may have any of the keys missing resulting in 
        # empty lists of indices

        for i in range(0,n):    
            metadata_idx += list(range(data_l1[i]+1,data_l2_begin[i]))
            if i < n-1:
                metadata_idx += list(range(data_l2_end[i]+1,data_l1[i+1]))

        metadata_idx += list(range(data_l2_end[n-1]+1,data_other[0]))
        metadata_idx += list(range(data_other[-1]+1,tmpdf.shape[0]))
        
        # copy metadata rows to its own dataframe
        tmpdf_md = tmpdf.loc[metadata_idx,:]
        # clean-up
        tmpdf_md.dropna(axis=1,how='all',inplace=True)
        tmpdf_md.dropna(axis=0,how='all',inplace=True)

        # purge metadata rows (copied above) from the data df
        tmpdf.drop(metadata_idx,inplace=True)
        # clean-up
        tmpdf.dropna(axis=1,how='all',inplace=True)
        tmpdf.dropna(axis=0,how='all',inplace=True)
        
        # collect l1 header names
        # needed because we don't know the order in which the l1 headers occur in data
        df_l1 = tmpdf.loc[data_l1]
        df_l1 = df_l1.loc[:,df_l1.notnull().any()]
        l1_hdrs = df_l1.T.unstack().tolist()
        l1_hdrs = [s for s in l1_hdrs if str(s) != 'nan']
        
        # drop all l1 headers
        # we will be using a single-level index for the final df
        # l1 headers will be used to populate a categorical var instead
        tmpdf.drop(data_l1,inplace=True)

        # create a new ddtaframe for each school type
        df_list = []
        for i in range(0,n):
    
            row_idx = list(range(data_l2_begin[i]+1,data_l2_end[i]+1))   
            col_idx = data_l2_begin[i]
            school_type = l1_hdrs[i]    
    
            df_list.append(extract_df(tmpdf,row_idx,col_idx,school_type))
    
            # if this the last of the school types we need to append
            # the aggregated lea rows. we do this as a separate df containing
            # data_other rows.
            if (i==n-1):
                row_idx = data_other
                df_list.append(extract_df(tmpdf,row_idx,col_idx,np.nan))
        
        # we have a df with all data for all school types including aggregated
        # rows at this point
        df_full = pd.concat(df_list,axis=0,ignore_index=True)
        
        # recode column names
        df_full.rename(columns=DATA_L2_HDR_DICT,inplace=True)
        # recode school_type
        df_full['school_type'] = df_full['school_type'].map(DATA_L1_HDR_DICT)
        # recode other fields
        cond = df_full['index'].isin(data_l2_end + data_other)
        df_full.loc[cond,'school_name'] = df_full[cond]['school_code'].map(DATA_SUM_DICT)
        df_full.loc[cond,'school_code'] = INVALID_SCHOOL_CODE
        cond = df_full['index'].isin(data_other)
        df_full.loc[cond,'school_type'] = ALL_SCHOOL_TYPE
        
        df_full.drop(['index'],axis=1,inplace=True)
        # re-arrange cols to original order
        df_full = df_full[list(DATA_L2_HDR_DICT.values()) + ['school_type']]
        
        #
        # METADATA
        #
        
        # add appropriate prefix and suffix to metadata keys 
        md_keys = ['   ' + s + ':' for s in METADATA_KEYS]
        # get indices for rows where the metadata keywords are present
        md_idx = tmpdf_md.index[tmpdf_md[tmpdf_md.isin(md_keys)].notnull().any(axis=1)].tolist()
        
        # extract non-null cols only for those rows containing metadata keys
        tmpdf_md = tmpdf_md.loc[md_idx,:]
        tmpdf_md.dropna(axis=1,how='all',inplace=True)
        tmpdf_md.dropna(axis=0,how='all',inplace=True)
        tmpdf_md.columns = list(range(0,tmpdf_md.shape[1]))

        # extract metadata keys
        md_keys = list(tmpdf_md.loc[:,METADATA_KEY_COLS].unstack().values)
        md_keys = list(map(str.strip,md_keys))
        md_keys = list(map(str.lower,md_keys))
        md_keys = [s.replace(' ','_') for s in md_keys]
        md_keys = [s[:-1] for s in md_keys]
        
        # extract metadata values
        md_vals = list(tmpdf_md.loc[:,METADATA_VAL_COLS].unstack().values)
        md_vals = [s.lower() if isinstance(s, str) else s for s in md_vals]

        md_dict = dict(zip(md_keys, md_vals))
        
        # store only at the end when we have successfully completed all steps
        # for both data and metadata
        self.d_df = df_full
        self.md_dict = md_dict
                
    except Exception as e:
        raise e

In [6]:
class mcXLSchoolDistInput(mcSchoolDistInput):
    """
    Implementation for MealsCount Excel format school district input. Expects input 
    to be a file stored on the backend.
    """        
    
    def __init__(self, datafile):        
        mcSchoolDistInput.__init__(self)
        self.__datafile = datafile
        try:
            self.__parse(self.__datafile)
        except Exception as e:
            raise e
    
    def to_frame(self):
        return self.d_df
    
    def metadata(self):
        return self.md_dict
    
    __parse = parseXL           
        

### Usage    
  
Below code fragments demonstrate the usage of the above functionality.    

In [7]:
CWD = os.getcwd()

DATADIR = "data"
DATAFILE = "calpads_sample_data.xlsx"

In [8]:
data_in = mcXLSchoolDistInput(os.path.join(DATADIR,DATAFILE))
df = data_in.to_frame()
df.head()

,school_code,school_name,total_enrolled,frpm,foster,homeless,migrant,direct_cert,frpm_nodup,el,frpm_el_nodup,school_type
0,1000001,School NC01,37,4,27,0,0,6,29,5,30,non-charter
1,1000002,School NC02,1111,503,2,7,0,215,527,122,556,non-charter
2,1000003,School NC03,2332,897,2,14,0,440,979,169,1037,non-charter
3,1000004,School NC03,854,733,6,10,1,361,765,296,792,non-charter
4,1000005,School NC04,1628,1119,7,17,0,640,1239,474,1333,non-charter


In [9]:
df.tail()

,school_code,school_name,total_enrolled,frpm,foster,homeless,migrant,direct_cert,frpm_nodup,el,frpm_el_nodup,school_type
31,2000001,School C01,460,321,0,0,0,182,332,192,364,charter
32,2000002,School C02,420,313,0,2,0,191,323,253,363,charter
33,2000003,School C03,246,119,0,11,0,85,152,187,222,charter
34,9999999,total,1126,753,0,13,0,458,807,632,949,charter
35,9999999,total,40319,21800,111,528,19,12279,23704,8673,25663,lea


In [10]:
df[df['school_name']=='total']

,school_code,school_name,total_enrolled,frpm,foster,homeless,migrant,direct_cert,frpm_nodup,el,frpm_el_nodup,school_type
30,9999999,total,39193,21047,111,515,19,11821,22897,8041,24714,non-charter
34,9999999,total,1126,753,0,13,0,458,807,632,949,charter
35,9999999,total,40319,21800,111,528,19,12279,23704,8673,25663,lea


In [11]:
metadata = data_in.metadata()
print(metadata)

{'academic_year': '2017-2018', 'view': 'ods', 'as_of': datetime.datetime(2018, 3, 12, 0, 0), 'gender': 'all', 'school_type': 'all', 'school': 'all', 'user_id': nan, 'created_date': '03-12-2018', 'lea': 'somecity union high'}


### TODO  
  
* Refactor ParseXL functionality  
* Add API for error status ?
* Profile parser for large datasets ?

In [12]:
df["isp_percent"] = df.frpm_el_nodup / df.total_enrolled
df['isp_students'] = df.frpm_el_nodup
df['total_enrollment'] = df.total_enrolled

In [13]:
df.sort_values(by=['isp_percent'], inplace=True, ascending=False)

In [14]:
data = df[["total_enrolled","frpm_el_nodup"]].cumsum()

In [15]:
data['isp_percentage'] = data.frpm_el_nodup / data.total_enrolled

In [16]:
one_hundred_percent_indexes = data[data.isp_percentage>.825].index

In [17]:
one_hundred_percent_indexes

Int64Index([17, 25,  3, 33, 15,  5, 10, 32, 24, 13, 28, 34, 26,  4,  0,  6, 31,
            22, 11, 20, 16,  7, 27, 14, 29],
           dtype='int64')

In [18]:
mask = df.mask(data.isp_percentage > .825)
mask_cumsum = mask[["total_enrolled","frpm_el_nodup"]].cumsum()
mask_cumsum['isp_percent'] = mask.frpm_el_nodup / mask_cumsum.total_enrolled
min_percent_schools = mask[mask_cumsum.isp_percent> .30]
min_percent_schools

,school_code,school_name,total_enrolled,frpm,foster,homeless,migrant,direct_cert,frpm_nodup,el,frpm_el_nodup,school_type,isp_percent,isp_students,total_enrollment
35,9999999,total,40319,21800,111,528,19,12279,23704,8673,25663,lea,0.636499,25663,40319
30,9999999,total,39193,21047,111,515,19,11821,22897,8041,24714,non-charter,0.630572,24714,39193


In [19]:
mask

,school_code,school_name,total_enrolled,frpm,foster,homeless,migrant,direct_cert,frpm_nodup,el,frpm_el_nodup,school_type,isp_percent,isp_students,total_enrollment
17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
data['group']=np.NaN
data.at[data[data.isp_percentage>.825].index, 'group']="Group 1"

/Users/jlangley/miniconda3/envs/py36/lib/python3.6/site-packages/ipykernel/zmqshell.py:533: PerformanceWarning: indexing past lexsort depth may impact performance.
  return super(ZMQInteractiveShell, self).run_cell(*args, **kwargs)


In [35]:
data.group

group    govt_funding_level
Group 1  1                     Group 1
         1                     Group 1
         1                     Group 1
         1                     Group 1
         1                     Group 1
         1                     Group 1
         1                     Group 1
         1                     Group 1
         1                     Group 1
         1                     Group 1
         1                     Group 1
         1                     Group 1
         1                     Group 1
         1                     Group 1
         1                     Group 1
         1                     Group 1
         1                     Group 1
         1                     Group 1
         1                     Group 1
         1                     Group 1
         1                     Group 1
         1                     Group 1
         1                     Group 1
         1                     Group 1
         1                     Group

In [36]:
mask_cumsum['isp_percent'][mask_cumsum.isp_percent > .11].tail(1)

Series([], Name: isp_percent, dtype: object)

In [37]:

#rates = config.cep_rates(region=kwargs['state'])
data['group'] = np.NaN

# gives an index list of values where ISP_percent is > .825
data['isp_percent'] = data.frpm_el_nodup / data.total_enrolled
data.sort_values(by=['isp_percent'], inplace=True, ascending=False)

top_group = data[["total_enrolled", "frpm_el_nodup"]].cumsum()
top_group['isp_percent'] = top_group.frpm_el_nodup / top_group.total_enrolled
one_hundred_percent_funding_indexes = top_group[top_group.isp_percent > .825].index

len(one_hundred_percent_funding_indexes)

26

In [38]:
data.at[one_hundred_percent_funding_indexes, 'group'] = "Group 1"

greater_than_minimum = data.mask(data.isp_percentage > .6)
mask_cumsum = greater_than_minimum[["total_enrolled","frpm_el_nodup"]].cumsum()
mask_cumsum['isp_percent'] = mask_cumsum.frpm_el_nodup / mask_cumsum.total_enrolled
min_percent_schools = greater_than_minimum[mask_cumsum.isp_percent > .6].index
data.at[min_percent_schools, 'group'] = "Group 2"

group_2_percent = mask_cumsum['isp_percent'][mask_cumsum.isp_percent > .6].tail(1)

data['govt_funding_level'] = np.where(data.isp_percent * 1.3 > 1, 1, data.isp_percent * 1.3)
data.sort_values(['isp_percent', 'frpm_el_nodup'],
                 ascending=[False, False],
                 inplace=True)

/Users/jlangley/miniconda3/envs/py36/lib/python3.6/site-packages/ipykernel/zmqshell.py:533: PerformanceWarning: indexing past lexsort depth may impact performance.
  return super(ZMQInteractiveShell, self).run_cell(*args, **kwargs)


In [39]:
data[(data.isp_percentage > .6) & (data.isp_percentage<.825)]

total_enrolled frpm_el_nodup isp_percentage  \
group   govt_funding_level                                               
Group 1 1                           64779         45914       0.708779   
NaN     1                          103972         70628       0.679298   
        1                          106061         71919       0.678091   
        1                          107172         72475       0.676249   
        1                          109504         73512       0.671318   
        1                          109598         73548       0.671071   
        1                          112001         74405       0.664324   
        1                          113713         74972       0.659309   
        1                          116218         75729       0.651612   
        1                          117926         76207       0.646227   
        1                          120957         76989       0.636499   

                           isp_percent    group govt_funding_level  
group   govt_funding_level                                          
Group 1 1                     0.708779  Group 1           0.921413  
NaN     1                     0.679298      NaN           0.883088  
        1                     0.678091      NaN           0.881518  
        1                     0.676249      NaN           0.879124  
        1                     0.671318      NaN           0.872713  
        1                     0.671071      NaN           0.872392  
        1                     0.664324      NaN           0.863622  
        1                     0.659309      NaN           0.857102  
        1                     0.651612      NaN           0.847095  
        1                     0.646227      NaN           0.840095  
        1                     0.636499      NaN           0.827449

In [40]:
data

total_enrolled frpm_el_nodup isp_percentage  \
group   govt_funding_level                                               
Group 1 1                             792           754        0.95202   
        1                            1435          1356       0.944948   
        1                            2289          2148       0.938401   
        1                            2535          2370       0.934911   
        1                            3393          3126       0.921309   
        1                            4249          3872       0.911273   
        1                            5216          4708       0.902607   
        1                            5636          5071       0.899752   
        1                            8078          7164       0.886853   
        1                            8867          7840       0.884177   
        1                           11516         10104       0.877388   
        1                           12642         11053       0.874308   
        1                           14454         12580       0.870347   
        1                           16082         13913       0.865129   
        1                           16119         13943       0.865004   
        1                           18496         15835       0.856131   
        1                           18956         16199       0.854558   
        1                           19322         16485       0.853173   
        1                           20338         17267       0.849002   
        1                           20469         17367       0.848454   
        1                           22264         18734       0.841448   
        1                           22341         18789        0.84101   
        1                           22365         18806       0.840867   
        1                           23953         19909       0.831169   
        1                           24460         20251       0.827923   
        1                           64779         45914       0.708779   
NaN     1                          103972         70628       0.679298   
        1                          106061         71919       0.678091   
        1                          107172         72475       0.676249   
        1                          109504         73512       0.671318   
        1                          109598         73548       0.671071   
        1                          112001         74405       0.664324   
        1                          113713         74972       0.659309   
        1                          116218         75729       0.651612   
        1                          117926         76207       0.646227   
        1                          120957         76989       0.636499   

                           isp_percent    group govt_funding_level  
group   govt_funding_level                                          
Group 1 1                      0.95202  Group 1                  1  
        1                     0.944948  Group 1                  1  
        1                     0.938401  Group 1                  1  
        1                     0.934911  Group 1                  1  
        1                     0.921309  Group 1                  1  
        1                     0.911273  Group 1                  1  
        1                     0.902607  Group 1                  1  
        1                     0.899752  Group 1                  1  
        1                     0.886853  Group 1                  1  
        1                     0.884177  Group 1                  1  
        1                     0.877388  Group 1                  1  
        1                     0.874308  Group 1                  1  
        1                     0.870347  Group 1                  1  
        1                     0.865129  Group 1                  1  
        1                     0.865004  Group 1                  1  
        1                     0.856131  G

In [41]:
data.set_index(['group','govt_funding_level'], inplace=True)

In [42]:
data

total_enrolled frpm_el_nodup isp_percentage  \
group   govt_funding_level                                               
Group 1 1.000000                      792           754        0.95202   
        1.000000                     1435          1356       0.944948   
        1.000000                     2289          2148       0.938401   
        1.000000                     2535          2370       0.934911   
        1.000000                     3393          3126       0.921309   
        1.000000                     4249          3872       0.911273   
        1.000000                     5216          4708       0.902607   
        1.000000                     5636          5071       0.899752   
        1.000000                     8078          7164       0.886853   
        1.000000                     8867          7840       0.884177   
        1.000000                    11516         10104       0.877388   
        1.000000                    12642         11053       0.874308   
        1.000000                    14454         12580       0.870347   
        1.000000                    16082         13913       0.865129   
        1.000000                    16119         13943       0.865004   
        1.000000                    18496         15835       0.856131   
        1.000000                    18956         16199       0.854558   
        1.000000                    19322         16485       0.853173   
        1.000000                    20338         17267       0.849002   
        1.000000                    20469         17367       0.848454   
        1.000000                    22264         18734       0.841448   
        1.000000                    22341         18789        0.84101   
        1.000000                    22365         18806       0.840867   
        1.000000                    23953         19909       0.831169   
        1.000000                    24460         20251       0.827923   
        0.921413                    64779         45914       0.708779   
NaN     0.883088                   103972         70628       0.679298   
        0.881518                   106061         71919       0.678091   
        0.879124                   107172         72475       0.676249   
        0.872713                   109504         73512       0.671318   
        0.872392                   109598         73548       0.671071   
        0.863622                   112001         74405       0.664324   
        0.857102                   113713         74972       0.659309   
        0.847095                   116218         75729       0.651612   
        0.840095                   117926         76207       0.646227   
        0.827449                   120957         76989       0.636499   

                           isp_percent  
group   govt_funding_level              
Group 1 1.000000               0.95202  
        1.000000              0.944948  
        1.000000              0.938401  
        1.000000              0.934911  
        1.000000              0.921309  
        1.000000              0.911273  
        1.000000              0.902607  
        1.000000              0.899752  
        1.000000              0.886853  
        1.000000              0.884177  
        1.000000              0.877388  
        1.000000              0.874308  
        1.000000              0.870347  
        1.000000              0.865129  
        1.000000              0.865004  
        1.000000              0.856131  
        1.000000              0.854558  
        1.000000              0.853173  
        1.000000              0.849002  
        1.000000              0.848454  
        1.000000              0.841448  
        1.000000               0.84101  
        1.000000              0.840867  
        1.000000              0.831169  
        1.000000              0.827923  
        0.921413              0.708779  
NaN     0.883088              0.679298  
        0.881518         

In [48]:
data.mask(top_group.isp_percent > .7)

Exception: cannot handle a non-unique multi-index!

In [54]:
    data = pd.DataFrame(
        {
        "schoolnames":[str(x) for x in range(1000)],
        "isp_students":[round(x*2.5) for x in range(1000)],
        "total_enrollment": 3000,
    })
    config = pd.Series()
    config.max_cep_thold_pct = .625
    config.min_cep_thold_pct = .4


In [55]:
data

,isp_students,schoolnames,total_enrollment
0,0,0,3000
1,2,1,3000
2,5,2,3000
3,8,3,3000
4,10,4,3000
5,12,5,3000
6,15,6,3000
7,18,7,3000
8,20,8,3000
9,22,9,3000
